In [3]:
import numpy as np
import pandas as pd
import datetime
from skimpy import skim



In [ ]:
pd.read_csv()


In [12]:
df.head()

,Activity ID,Activity Date,Activity Name,Activity Type,Distance,Moving Time,Average Heart Rate,Calories,Average Temperature
0,1927719,"Sep 28, 2011, 4:37:41 AM","09/28/2011 Istres, Provence-Alpes-Côte d'Azur,...",Run,3.54,1253.0,NaN,371.595459,NaN
1,1927721,"Sep 30, 2011, 4:35:32 AM","09/30/2011 Istres, Provence-Alpes-Côte d'Azur,...",Run,2.44,746.0,NaN,259.392212,NaN
2,1927722,"Sep 26, 2011, 4:34:33 AM","09/26/2011 Istres, Provence-Alpes-Côte d'Azur,...",Run,3.52,1252.0,NaN,373.191833,NaN
3,1927723,"Oct 10, 2011, 6:42:14 AM","10/10/2011 Cambridge, Cambridgeshire, United K...",Run,6.70,2222.0,NaN,693.651062,NaN
4,1927724,"Sep 23, 2011, 4:35:12 AM","09/23/2011 Istres, Provence-Alpes-Côte d'Azur,...",Run,3.63,1334.0,NaN,379.204193,NaN


In [13]:
# perform cleanup functions such as column names and data conversions
# not necessary if using a local copy that has already been cleaned
df = strava_prepare.typ_clean(df)

In [14]:
df.head()

,activity_id,activity_date,activity_name,distance,moving_time,average_heart_rate,calories,average_temperature
0,1927719,"Sep 28, 2011, 4:37:41 AM","09/28/2011 Istres, Provence-Alpes-Côte d'Azur,...",2.20,0:20:53,NaN,371.595459,NaN
1,1927721,"Sep 30, 2011, 4:35:32 AM","09/30/2011 Istres, Provence-Alpes-Côte d'Azur,...",1.52,0:12:26,NaN,259.392212,NaN
2,1927722,"Sep 26, 2011, 4:34:33 AM","09/26/2011 Istres, Provence-Alpes-Côte d'Azur,...",2.19,0:20:52,NaN,373.191833,NaN
3,1927723,"Oct 10, 2011, 6:42:14 AM","10/10/2011 Cambridge, Cambridgeshire, United K...",4.16,0:37:02,NaN,693.651062,NaN
4,1927724,"Sep 23, 2011, 4:35:12 AM","09/23/2011 Istres, Provence-Alpes-Côte d'Azur,...",2.26,0:22:14,NaN,379.204193,NaN


In [16]:
df.value_counts(dropna=False).sum()

1937

In [ ]:
# save any updates to csv file
df.to_csv('strava.csv', index=False)
